In [ ]:
class MyPipeline:
    pass

def my_function(name: str, age: int):
    return {"message": f"Hello {name}, you are {age} years old."}


In [ ]:
from fastapi import FastAPI, APIRouter, Depends, Body
from typing import Callable, get_type_hints
import inspect

class APIBuilder:
    
    def __init__(self):
        self.router:APIRouter = None  
        
    
    @staticmethod
    def init():
        __api = APIBuilder()
        __api.router = APIRouter()
        return __api


    # app = FastAPI()

    def add_endpoint_to_router(self, func: Callable, mode: str = 'query'):
        """
        Dynamically adds an endpoint to a FastAPI router based on the provided function.
        The function's name is used as the endpoint name, and its parameters are used as query parameters or a request body.

        Args:
        - router: The FastAPI APIRouter to add the endpoint to.
        - func: The function to create the endpoint from.
        - mode: 'query' for query parameters, 'body' for a request body. Defaults to 'query'.
        """
        endpoint_path = f"/{func.__name__}"  # Use function name as the endpoint path
        type_hints = get_type_hints(func)
        
        async def endpoint(**kwargs):
            return func(**kwargs)

        if mode == 'query':
            # Use function parameters as query parameters
            params = inspect.signature(func).parameters
            dependencies = [
                Depends(lambda name=param.name, annotation=param.annotation: Body(..., example="example")) if mode == 'body' else None
                for param in params.values()
            ]
            self.router.add_api_route(endpoint_path, endpoint, methods=["GET"], dependencies=dependencies)
        elif mode == 'body':
            # Use a single request body object that matches the function's parameters
            async def endpoint(body: dict = Body(...)):
                return func(**body)
            self.router.add_api_route(endpoint_path, endpoint, methods=["POST"])

    # Example function to create an endpoint from

    # Adding the endpoint to the router
    # add_endpoint_to_router(router, my_function, mode='query')

    # Including the router in the FastAPI app
    # app.include_router(router)
